## <center>Partitioning: Divide and Conquer</center>
### <center> Linh B. Ngo </center>
### <center> CPSC 4770-6770 </center>

#### <center> Partitioning </center>

Partitioning simply divides the problem into parts and then compute the parts and combine results

- The basis of all parallel programming, in one form or another. 

- Pleasantly parallel used partitioning without any interaction between the parts.

- Most partitioning  formulation require the results of the parts to be combined to obtain the desired results. 

- Partitioning can be applied to the program data. This is call data partitioning or domain decomposition.

- Partitioning can also be applied to the functions of a program. This is called functional decomposition. 

#### <center> Divide and Conquer </center>

- Characterized by dividing problem into sub-problems of same form as larger problem. Further divisions into still smaller sub-problems, usually done by recursion.

- Recursive divide and conquer amenable to parallelization because separate processes can be used for divided pairs. Also usually data is naturally localized.

<center> <img src="pictures/07/dc01.png" width="700"/> 
<sub>Wilkinson, Barry, and Michael Allen. Parallel programming. 2nd Ed. 2003. </sub>
</center>

<center> <img src="pictures/07/dc02.png" width="700"/> 
<sub>Wilkinson, Barry, and Michael Allen. Parallel programming. 2nd Ed. 2003. </sub>
</center>

In [8]:
%%writefile codes/mpi4py/divide.py
#!/usr/bin/env python
# divide.py
from mpi4py import MPI
import math
comm = MPI.COMM_WORLD
rank = comm.Get_rank(); size = comm.Get_size()
for i in range(0, int(math.log2(size))):
    distance = int(math.pow(2, i))
    if ( rank < int(math.pow(2, i)) ):
        print ("Iteration ", i, ": sender ", rank, 
               " sends to ", rank + distance)
    if (rank >= int(math.pow(2, i)) and rank < int(math.pow(2,i+1))):
        print ("Iteration ", i, ": receiver ", rank, 
               "receives from ", rank - distance)
    #am I sender or receiver?
    # who am I sending/receiving to/from?    

Overwriting codes/mpi4py/divide.py


In [1]:
!chmod 755 codes/mpi4py/divide.py
!module load gcc/5.3.0 openmpi/1.10.3; mpirun -np 8 codes/mpi4py/divide.py

Iteration  0 : sender  0  sends to  1
Iteration  1 : sender  0  sends to  2
Iteration  2 : sender  0  sends to  4
Iteration  0 : receiver  1 receives from  0
Iteration  1 : sender  1  sends to  3
Iteration  2 : sender  1  sends to  5
Iteration  1 : receiver  2 receives from  0
Iteration  2 : sender  2  sends to  6
Iteration  1 : receiver  3 receives from  1
Iteration  2 : sender  3  sends to  7
Iteration  2 : receiver  4 receives from  0
Iteration  2 : receiver  5 receives from  1
Iteration  2 : receiver  6 receives from  2
Iteration  2 : receiver  7 receives from  3


<center> <img src="pictures/07/dc03.png" width="700"/> 
<sub>Wilkinson, Barry, and Michael Allen. Parallel programming. 2nd Ed. 2003. </sub>
</center>

In [2]:
%%writefile codes/mpi4py/conquer.py
#!/usr/bin/env python
# conquer.py
from mpi4py import MPI
import math
comm = MPI.COMM_WORLD
rank = comm.Get_rank(); size = comm.Get_size()
for i in range(int(math.log2(size)) - 1, -1, -1):
    distance = int(math.pow(2, i))
    if ( rank < int(math.pow(2, i)) ):
        print ("Iteration ", i, ": receiver ", rank, 
               " receives from ", rank + distance)
    if (rank >= int(math.pow(2, i)) and rank < int(math.pow(2,i+1))):
        print ("Iteration ", i, ": sender ", rank, 
               "sends to ", rank - distance)
    #am I sender or receiver?
    # who am I sending/receiving to/from?    

Overwriting codes/mpi4py/conquer.py


In [3]:
!chmod 755 codes/mpi4py/conquer.py
!module load gcc/5.3.0 openmpi/1.10.3; mpirun -np 8 codes/mpi4py/conquer.py

Iteration  2 : sender  5 sends to  1
Iteration  2 : sender  6 sends to  2
Iteration  2 : sender  7 sends to  3
Iteration  2 : receiver  0  receives from  4
Iteration  1 : receiver  0  receives from  2
Iteration  0 : receiver  0  receives from  1
Iteration  2 : receiver  1  receives from  5
Iteration  1 : receiver  1  receives from  3
Iteration  0 : sender  1 sends to  0
Iteration  2 : receiver  2  receives from  6
Iteration  1 : sender  2 sends to  0
Iteration  2 : receiver  3  receives from  7
Iteration  1 : sender  3 sends to  1
Iteration  2 : sender  4 sends to  0


**Many sorting algorithms can be parallelized by partitioning using
divide and conquer**

#### <center> Bucket Sort </center>

<center> <img src="pictures/07/bucketsort1.png" width="700"/> 
<sub>Wilkinson, Barry, and Michael Allen. Parallel programming. 2nd Ed. 2003. </sub>
</center>

** Simple approach to parallel bucket sort **

<center> <img src="pictures/07/bucketsort2.png" width="700"/> 
<sub>Wilkinson, Barry, and Michael Allen. Parallel programming. 2nd Ed. 2003. </sub>
</center>

- Bcast data
- Sort only those elements that fit in local interval bucket (determined by rank)
- Gather sorted bucket

**Uppercase collective: MPI.Scatterv**

Comm.Scatterv([sendbuf, tuple_int sendcounts, tuple_int displacements, MPI_Datatype sendtype], recvbuf, root=0)

Parameters:	
- Comm (MPI comm) – communicator across which to scatter
- sendbuf (choice) – buffer
- sendcounts (tuple_int) – number of elements to send to each process (one integer for each process)
- displacements (tuple_int) – number of elements away from the first element in the array at which to begin the new, segmented array
- sendtype (MPI_Datatype) – MPI datatype of the buffer being sent (choice of sendbuf)
- recvbuf (choice) – buffer in which to receive the sendbuf
- root (int) – process from which to scatter

In [5]:
%%writefile codes/mpi4py/scatterv.py
#!/usr/bin/env python
# scatterv.py
# Run with 3 processes
import numpy
from mpi4py import MPI
comm = MPI.COMM_WORLD; rank = comm.Get_rank()
if rank == 0:
    x = numpy.linspace(0,100,11)
    print (x)
else:
    x = None
if rank == 2:
    xlocal = numpy.zeros(9)
else:
    xlocal = numpy.zeros(1)
comm.Scatterv([x,(1,1,9),(0,1,2),MPI.DOUBLE],xlocal)
print ("process " + str(rank) + " has " +str(xlocal))
#if (rank == 0):
  #  print(x)

Overwriting codes/mpi4py/scatterv.py


In [6]:
!chmod 755 codes/mpi4py/scatterv.py
!module load gcc/5.3.0 openmpi/1.10.3; mpirun -np 3 codes/mpi4py/scatterv.py

--------------------------------------------------------------------------
The library attempted to open the following supporting CUDA libraries, 
but each of them failed.  CUDA-aware support is disabled.
libcuda.so.1: cannot open shared object file: No such file or directory
libcuda.dylib: cannot open shared object file: No such file or directory
/usr/lib64/libcuda.so.1: cannot open shared object file: No such file or directory
/usr/lib64/libcuda.dylib: cannot open shared object file: No such file or directory
If you are not interested in CUDA-aware support, then run with 
--mca mpi_cuda_support 0 to suppress this message.  If you are interested
in CUDA-aware support, then try setting LD_LIBRARY_PATH to the location
of libcuda.so.1 to get passed this issue.
--------------------------------------------------------------------------
process 1 has [ 10.]
process 2 has [  20.   30.   40.   50.   60.   70.   80.   90.  100.]
[   0.   10.   20.   30.   40.   50.   60.   70.   80.   90.  100

**Uppercase collective: MPI.Gatherv**

Comm.Gatherv(sendbuf, [recvbuf, tuple_int sendcounts, tuple_int displacements, MPI_Datatype sendtype], root=0)

Parameters:	
- Comm (MPI comm) – communicator across which to scatter
- sendbuf (choice) – buffer
- recvbuf (choice) – buffer in which to receive the sendbuf
- sendcounts (tuple_int) – number of elements to receive from each process (one integer for each process)
- displacements (tuple_int) – number of elements away from the first element in the receiving array at which to begin appending the segmented array
- sendtype (MPI_Datatype) – MPI datatype of the buffer being sent (choice of sendbuf)
- root (int) – process from which to scatter

In [14]:
%%writefile codes/mpi4py/gatherv.py
#!/usr/bin/env python
# gatherv.py
# Run with 3 processes
import numpy
from mpi4py import MPI
comm = MPI.COMM_WORLD; rank = comm.Get_rank()
if rank == 0:
    x = numpy.linspace(0,80,9)
    print (x)
else:
    x = None
if rank == 1:
    xlocal = numpy.zeros(7)
else:
    xlocal = numpy.zeros(1)
comm.Scatterv([x,(1,7,1),(0,1,8),MPI.DOUBLE],xlocal); 
if rank == 0:
    xGathered = numpy.zeros(9)
else:
    xGathered = None
comm.Gatherv(xlocal,[xGathered,(1,7,1),(0,1,1),MPI.DOUBLE])
print (" process ", rank, " has ", xlocal); 
if (rank == 0):
    print ("process " + str(rank) + " has " +str(xGathered))

Overwriting codes/mpi4py/gatherv.py


In [16]:
!chmod 755 codes/mpi4py/gatherv.py
!module load gcc/5.3.0 openmpi/1.10.3; mpirun -np 3 --mca mpi_cuda_support 0 codes/mpi4py/gatherv.py

[  0.  10.  20.  30.  40.  50.  60.  70.  80.]
 process  0  has  [ 0.]
process 0 has [  0.  80.  20.  30.  40.  50.  60.  70.   0.]
 process  1  has  [ 10.  20.  30.  40.  50.  60.  70.]
 process  2  has  [ 80.]


**Parallel Bucket Sort version 1**

In [17]:
%%writefile codes/mpi4py/bucket1.py
#!/usr/bin/env python
# bucket1.py
import numpy as np
from mpi4py import MPI
comm = MPI.COMM_WORLD

rank = comm.Get_rank(); size = comm.Get_size(); N = 16

unsorted = np.zeros(N, dtype="int")
final_sorted = np.zeros(N, dtype="int")

if rank == 0:
    unsorted = np.random.randint(low=0,high=N,size=N)
    
comm.Bcast(unsorted)
local_min = rank * N / size
local_max = (rank + 1) * N / size

#generic form:
# local_min = rank * (b - a) / size
# local_max = (rank + 1) * (b - a) / size

print("At rank ", rank, ": ", local_min, local_max)

local_bucket = unsorted[np.logical_and(unsorted >= local_min, 
                                       unsorted < local_max)]
local_sorted = np.sort(local_bucket)

arr_sendcount = np.zeros(size, dtype="int")
sendcount = np.array([local_sorted.size], dtype="int")

comm.Gather(sendcount, arr_sendcount, root=0)
comm.Bcast(arr_sendcount, root = 0)

dispcount = np.zeros(size, dtype="int")
dispcount[0] = 0
for i in range(1,size):
    dispcount[i] = dispcount[i-1] + arr_sendcount[i-1]
    
comm.Gatherv(local_sorted, [final_sorted, 
                            tuple(arr_sendcount), 
                            tuple(dispcount), 
                            MPI.DOUBLE])

if (rank == 0):
    print ("At root: ", unsorted)
    print ("At root: ", final_sorted)
print("Rank", rank, ": ", local_bucket)
print("Rank", rank, ": ", local_sorted)

Overwriting codes/mpi4py/bucket1.py


In [18]:
!chmod 755 codes/mpi4py/bucket1.py
!module load gcc/5.3.0 openmpi/1.10.3; mpirun -np 4 --mca mpi_cuda_support 0  codes/mpi4py/bucket1.py

At rank  0 :  0.0 4.0
At root:  [15  8  5  1 10 11 15 10  7 12 11  4  1 11 12 11]
At root:  [ 1  1  4  5  7  8 10 10 11 11 11 11 12 12 15 15]
Rank 0 :  [1 1]
Rank 0 :  [1 1]
At rank  1 :  4.0 8.0
Rank 1 :  [5 7 4]
At rank  2 :  8.0 12.0
Rank 2 :  [ 8 10 11 10 11 11 11]
Rank 2 :  [ 8 10 10 11 11 11 11]
At rank  3 :  12.0 16.0
Rank 3 :  [15 15 12 12]
Rank 3 :  [12 12 15 15]
Rank 1 :  [4 5 7]


<center> <img src="pictures/07/bucketsort3.png" width="700"/> 
<sub>Wilkinson, Barry, and Michael Allen. Parallel programming. 2nd Ed. 2003. </sub>
</center>

<center> <img src="pictures/07/all2all.png" width="700"/> 
<sub>Wilkinson, Barry, and Michael Allen. Parallel programming. 2nd Ed. 2003. </sub>
</center>

<center> <img src="pictures/07/all2all_2.png" width="700"/> 
<sub>Wilkinson, Barry, and Michael Allen. Parallel programming. 2nd Ed. 2003. </sub>
</center>

**Uppercase collective: MPI.Alltoallv**

Comm.Alltoallv([sendbuf, tuple_int sendcounts, tuple_int displacements, MPI_Datatype sendtype],[recvbuf, tuple_int recvcounts, tuple_int displacements, MPI_Datatype sendtype])

Parameters:	
- Comm (MPI comm) – communicator across which to scatter
- sendbuf (choice) – buffer
- recvbuf (choice) – buffer in which to receive the sendbuf
- sendcounts (tuple_int) – number of elements to send to each process (one integer for each process)
- displacements (tuple_int) – number of elements away from the first element in the sending array at which to begin sending the segmented array
- sendtype (MPI_Datatype) – MPI datatype of the buffer being sent (choice of sendbuf)
- recvcounts (tuple_int) – number of elements to receive from each process (one integer for each process)
- displacements (tuple_int) – number of elements away from the first element in the receiving array at which to begin appending the segmented array
- sendtype (MPI_Datatype) – MPI datatype of the buffer being receive (choice of recvbuf)

In [18]:
%%writefile codes/mpi4py/all2allv.py
#!/usr/bin/env python
# all2allv.py
import numpy
import random
from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank(); size = comm.Get_size(); N = 8
local_array = numpy.random.randint(low = 0, high = N, size = N)
print("Rank ", rank, "local_array: ", local_array)
#stackoverflow
def constrained_sum_sample_pos(n, total):
    dividers = sorted(random.sample(range(1, total), n - 1))
    return [a - b for a, b in zip(dividers + [total], [0] + dividers)]
sendcount = numpy.array(constrained_sum_sample_pos(size, N), dtype="int")
dispcount = numpy.zeros(size, dtype="int")
dispcount[0] = 0
for i in range(1,size):
    dispcount[i] = dispcount[i-1] + sendcount[i-1]    
print ("Rank ", rank, " sendcount: ", sendcount)
print ("Rank ", rank, " dispcount: ", dispcount)
recvcount = numpy.zeros(size, dtype="int")
comm.Alltoall(sendcount, recvcount)
print("Rank ", rank, " recvcount: ", recvcount)
disprecv = numpy.zeros(size, dtype="int")
disprecv[0] = 0
for i in range(1,size):
    disprecv[i] = disprecv[i-1] + recvcount[i-1]   
print("Rank ", rank, " disprecv: ", disprecv)
new_array = numpy.zeros(numpy.sum(recvcount), dtype="int")
comm.Alltoallv([local_array,tuple(sendcount),tuple(dispcount),MPI.DOUBLE],
               [new_array, tuple(recvcount), tuple(disprecv), MPI.DOUBLE])
print ("Rank ", rank, "new_array: ", new_array)
#print (local_array)

Overwriting codes/mpi4py/all2allv.py


In [19]:
!chmod 755 codes/mpi4py/all2allv.py
!module load gcc/5.3.0 openmpi/1.10.3; mpirun -np 4 --mca mpi_cuda_support 0  codes/mpi4py/all2allv.py

Rank  0 local_array:  [4 0 2 1 2 4 5 5]
Rank  0  sendcount:  [1 1 4 2]
Rank  0  dispcount:  [0 1 2 6]
Rank  1 local_array:  [4 2 7 4 5 1 6 0]
Rank  1  sendcount:  [3 1 1 3]
Rank  1  dispcount:  [0 3 4 5]
Rank  2 local_array:  [5 0 2 4 6 6 0 4]
Rank  2  sendcount:  [1 1 2 4]
Rank  2  dispcount:  [0 1 2 4]
Rank  3 local_array:  [4 1 3 4 7 2 5 0]
Rank  3  sendcount:  [2 1 4 1]
Rank  3  dispcount:  [0 2 3 7]
Rank  2  recvcount:  [4 1 2 4]
Rank  2  disprecv:  [0 4 5 7]
Rank  2 new_array:  [2 1 2 4 5 2 4 4 7 2 5]
Rank  3  recvcount:  [2 3 4 1]
Rank  3  disprecv:  [0 2 5 9]
Rank  3 new_array:  [5 5 1 6 0 6 6 0 4 0]
Rank  0  recvcount:  [1 3 1 2]
Rank  0  disprecv:  [0 1 4 5]
Rank  0 new_array:  [4 4 2 7 5 4 1]
Rank  1  recvcount:  [1 1 1 1]
Rank  1  disprecv:  [0 1 2 3]
Rank  1 new_array:  [0 4 0 3]


In [20]:
%%writefile codes/mpi4py/bucket2.py
#!/usr/bin/env python
# bucket2.py
import numpy as np
from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank(); size = comm.Get_size(); N = 16
unsorted = None
local_unsorted = np.zeros(int(N / size), dtype="int")
final_sorted = np.zeros(N, dtype="int")
if rank == 0:
    unsorted = np.random.randint(low=0,high=N,size=N)
    print ("Original unsorted data at process", rank, ": ", unsorted)
comm.Scatter(unsorted, local_unsorted, root = 0)
print ("Scattered data at process ", rank, ": ", local_unsorted)

local_buckets = np.full(shape=(int(N/size), size), fill_value=-1, dtype="int")
sendcount = np.zeros(size, dtype="int")
local_unsorted_bucketed = np.zeros(int(N / size), dtype="int")
count = 0
for i in range(0, size):    
    for j in range (0, local_unsorted.size):
        local_min = i * N/size
        local_max = (i + 1) * N / size  
        if ((local_unsorted[j] >= local_min) and (local_unsorted[j] < local_max)):
            local_buckets[i][sendcount[i]] = local_unsorted[j]
            sendcount[i] += 1
    if (sendcount[i] > 0):
        local_unsorted_bucketed[count:count + sendcount[i]] = local_buckets[i, 0:sendcount[i]]
        print ("Local_unsorted_bucketed at process ", rank, ": ", local_unsorted_bucketed)
        count += sendcount[i]

print ("Sendcount at process ", rank, ": ", sendcount)
print ("Bucket matrix at process ", rank, ": \n", local_buckets)

dispcount = np.zeros(size, dtype="int")
dispcount[0] = 0
for i in range(1,size):
    dispcount[i] = dispcount[i-1] + sendcount[i-1]    

recvcount = np.zeros(size, dtype="int")
comm.Alltoall(sendcount, recvcount)
print("Rank ", rank, " recvcount: ", recvcount)
disprecv = np.zeros(size, dtype="int")
disprecv[0] = 0
for i in range(1,size):
    disprecv[i] = disprecv[i-1] + recvcount[i-1]   
print("Rank ", rank, " disprecv: ", disprecv)
single_unsorted_bucket = np.zeros(np.sum(recvcount), dtype="int")
comm.Alltoallv([local_unsorted_bucketed,tuple(sendcount),tuple(dispcount),MPI.DOUBLE],
               [single_unsorted_bucket, tuple(recvcount), tuple(disprecv), MPI.DOUBLE])
print ("Rank ", rank, " single_unsorted_bucket: ", single_unsorted_bucket)
local_sorted = np.sort(single_unsorted_bucket)
arr_sendcount = np.zeros(size, dtype="int")
sendcount = np.array([local_sorted.size], dtype="int")
comm.Gather(sendcount, arr_sendcount, root=0)
comm.Bcast(arr_sendcount, root = 0)
dispcount = np.zeros(size, dtype="int")
dispcount[0] = 0
for i in range(1,size):
    dispcount[i] = dispcount[i-1] + arr_sendcount[i-1]
comm.Gatherv(local_sorted, [final_sorted, tuple(arr_sendcount), tuple(dispcount), MPI.DOUBLE])
if (rank == 0):
    print (final_sorted)

Overwriting codes/mpi4py/bucket2.py


In [21]:
!chmod 755 codes/mpi4py/bucket2.py
!module load gcc/5.3.0 openmpi/1.10.3; mpirun -np 4 --mca mpi_cuda_support 0  codes/mpi4py/bucket2.py

Original unsorted data at process 0 :  [ 3 11 14  7 15 15 12 13 12  8  1  7  5 11 13 14]
Scattered data at process  0 :  [ 3 11 14  7]
Local_unsorted_bucketed at process  0 :  [3 0 0 0]
Local_unsorted_bucketed at process  0 :  [3 7 0 0]
Scattered data at process  1 :  [15 15 12 13]
Local_unsorted_bucketed at process  1 :  [15 15 12 13]
Scattered data at process  2 :  [12  8  1  7]
Local_unsorted_bucketed at process  2 :  [1 0 0 0]
Local_unsorted_bucketed at process  2 :  [1 7 0 0]
Scattered data at process  3 :  [ 5 11 13 14]
Local_unsorted_bucketed at process  3 :  [5 0 0 0]
Local_unsorted_bucketed at process  3 :  [ 5 11  0  0]
Local_unsorted_bucketed at process  3 :  [ 5 11 13 14]
Sendcount at process  3 :  [0 1 1 2]
Bucket matrix at process  3 : 
 [[-1 -1 -1 -1]
 [ 5 -1 -1 -1]
 [11 -1 -1 -1]
 [13 14 -1 -1]]
Rank  3  recvcount:  [1 4 1 2]
Rank  3  disprecv:  [0 1 5 6]
Local_unsorted_bucketed at process  0 :  [ 3  7 11  0]
Local_unsorted_bucketed at process  0 :  [ 3  7 11 14]
Sendco

#### <center> N-Body Problem </center>

** Fundamental settings for most, if not all, of computational simulation problems: **

- Given a space
- Given a group of entities whose activities are (often) bounded within this space
- Given a set of equation that governs how these entities react to one another and to attributes of the containing space
- Simulate how these reactions impact all entities and the entire space overtime

- Computation requires parallelization
- Experimental spaces are simulated at massive scale (millions of entities)
- Individual time steps are significantly smaller than the total simulation time. 
- Time complexity can be reduced by approximating a cluster of distant bodies as a single distant body with mass sited at the center of the mass of the cluster

<center> <img src="pictures/07/mass-bodies.png" width="700"/> 
</center>

#### Barnes-Hut Algorithm (2-D)

Start with whole region in which one square contains the bodies (or particles).
- First, this cube is divided into four subregions.
- If a subregion contains no particles, it is deleted from further consideration.
- If a subregion contains one body, it is retained.
- If a subregion contains more than one body, it is recursively divided until every subregion contains one body.


- Create an quadtree – a tree with up to four edges from each node
- The leaves represent cells each containing one body.
- After the tree has been constructed, the total mass and center of mass of the subregion is stored at each node.


<center> <img src="pictures/07/barnes-hut.png" width="700"/> 
</center>

#### Orthogonal Recursive Bisection

- First, a vertical line found that divides area into two areas each with equal number of bodies. 
- For each area, a horizontal line found that divides it into two areas, each with equal number of bodies. 
- Repeated as required. 


<center> <img src="pictures/07/orthogonal.png" width="400"/> 
</center>